## 1. Complexity Hotspots

In [50]:
import pydriller
import subprocess
from pathlib import Path


# Path to the repository
repo_path = Path('./mastodon')

# Check if the repository exists
if repo_path.exists():
    print('Repository exists!')
else:
    # Clone the repository
    subprocess.call(['git', 'clone', 'https://github.com/mastodon/mastodon'])

path_to_repo = './mastodon'

Repository exists!


> Decide on the granularity of your analysis of software entities (e.g., source code files); describe why you selected this specific granularity.

File level, so that we don't need an additional parser. We only analyze source code files, so we focus on those with `.rb` extension.

> Create a list of all these entities, as they appear in the latest stable release of Mastodon (i.e., tag v3.5.3).



In [51]:
current_tag = 'v3.5.3'

gr = pydriller.Git(path_to_repo)
gr.checkout(current_tag)

In [52]:
import glob

entities_paths = glob.glob(path_to_repo + '/**/*.rb', recursive=True)

#remove initial path
entities_paths = [p[11:] for p in entities_paths]


entities_paths = [p for p in entities_paths if p.startswith('app')]


> Decide on the type of complexity you want to measure for your software entities and explain why you selected this type.

I go for LOC, because it's language independent and it's reasonably OK for complexity.

> Decide on a timeframe on which you want to base your analysis and explain the rationale of your choice.

I [see](https://github.com/ipfs/go-ipfs/tags) that go-ipfs undergoes a pretty sustained development, thus it is probably better not to use a too long timeframe. Since we also know exactly the tags, we can use those as key points in our analysis. I pick the first version of this release (i.e., tag `v3.5.0`), which corresponds to a timespan of ca. nine months.

In [53]:
start_tag = 'v3.4.0'

> For each entity in the system, measure its complexity and the number of changes (in the given timeframe). Merge these two pieces of information together to create a candidate list of problematic hotspots in mastodon/app.


In [54]:
def count_loc(file_path):
    loc = 0
    try:
        with open(path_to_repo + '/' + file_path) as f:
            for line in f:
                if not line.isspace():
                    loc += 1
    except FileNotFoundError:
        loc = 0
    return loc

In [55]:
metrics = dict()
loc = 'loc'
changes = 'changes'

for entity_path in entities_paths:
    metrics[entity_path] = {loc : count_loc(entity_path), changes : 0}

repo = pydriller.Repository(path_to_repo, from_tag = start_tag, to_tag = current_tag)

for commit in repo.traverse_commits():
    #print(commit)
    for m_file in commit.modified_files:
        if m_file.old_path and (m_file.new_path != m_file.old_path):
            print(m_file.old_path, "->", m_file.new_path)
        if m_file.new_path in entities_paths:
            metrics[m_file.new_path][changes] += 1

csv_like = [('fullpath',loc,changes)]

for fullpath,values in metrics.items():
    csv_like.append((fullpath,values[loc],values[changes]))

config/initializers/delivery_job.rb -> config/initializers/mail_delivery_job.rb
.github/ISSUE_TEMPLATE/support.md -> .github/ISSUE_TEMPLATE/3.support.md
.github/ISSUE_TEMPLATE/bug_report.md -> None
.github/ISSUE_TEMPLATE/feature_request.md -> None
spec/fixtures/xml/mastodon.atom -> None
lib/paperclip/media_type_spoof_detector_extensions.rb -> None
lib/paperclip/schema_extensions.rb -> None
lib/paperclip/validation_extensions.rb -> None
app/controllers/api/v1/admin/trends_controller.rb -> None
app/controllers/api/v1/trends_controller.rb -> None
app/models/trending_tags.rb -> None
app/views/admin/tags/_tag.html.haml -> None
app/views/admin/tags/index.html.haml -> None
app/views/admin_mailer/new_trending_tag.text.erb -> None
app/workers/scheduler/trending_tags_scheduler.rb -> app/workers/scheduler/trends/refresh_scheduler.rb
spec/controllers/api/v1/trends_controller_spec.rb -> None
spec/helpers/settings_helper_spec.rb -> spec/helpers/languages_helper_spec.rb
spec/models/trending_tags_spec

In [56]:
metrics

{'app/services/notify_service.rb': {'loc': 139, 'changes': 8},
 'app/services/fan_out_on_write_service.rb': {'loc': 115, 'changes': 6},
 'app/services/favourite_service.rb': {'loc': 36, 'changes': 2},
 'app/services/post_status_service.rb': {'loc': 162, 'changes': 5},
 'app/services/vote_service.rb': {'loc': 62, 'changes': 3},
 'app/services/after_block_service.rb': {'loc': 24, 'changes': 0},
 'app/services/follow_service.rb': {'loc': 71, 'changes': 1},
 'app/services/block_service.rb': {'loc': 21, 'changes': 0},
 'app/services/fetch_remote_status_service.rb': {'loc': 12, 'changes': 0},
 'app/services/tag_search_service.rb': {'loc': 74, 'changes': 0},
 'app/services/report_service.rb': {'loc': 56, 'changes': 3},
 'app/services/block_domain_service.rb': {'loc': 39, 'changes': 0},
 'app/services/deliver_to_device_service.rb': {'loc': 66, 'changes': 0},
 'app/services/unsuspend_account_service.rb': {'loc': 79, 'changes': 2},
 'app/services/batched_remove_status_service.rb': {'loc': 72, 'c

In [57]:
csv_like

[('fullpath', 'loc', 'changes'),
 ('app/services/notify_service.rb', 139, 8),
 ('app/services/fan_out_on_write_service.rb', 115, 6),
 ('app/services/favourite_service.rb', 36, 2),
 ('app/services/post_status_service.rb', 162, 5),
 ('app/services/vote_service.rb', 62, 3),
 ('app/services/after_block_service.rb', 24, 0),
 ('app/services/follow_service.rb', 71, 1),
 ('app/services/block_service.rb', 21, 0),
 ('app/services/fetch_remote_status_service.rb', 12, 0),
 ('app/services/tag_search_service.rb', 74, 0),
 ('app/services/report_service.rb', 56, 3),
 ('app/services/block_domain_service.rb', 39, 0),
 ('app/services/deliver_to_device_service.rb', 66, 0),
 ('app/services/unsuspend_account_service.rb', 79, 2),
 ('app/services/batched_remove_status_service.rb', 72, 1),
 ('app/services/verify_link_service.rb', 36, 1),
 ('app/services/remove_from_followers_service.rb', 19, 1),
 ('app/services/remove_status_service.rb', 115, 9),
 ('app/services/suspend_account_service.rb', 82, 0),
 ('app/serv

In [58]:
import pandas as pd

df_complexity = pd.DataFrame(csv_like[1:], columns=csv_like[0])
df_complexity

,fullpath,loc,changes
0,app/services/notify_service.rb,139,8
1,app/services/fan_out_on_write_service.rb,115,6
2,app/services/favourite_service.rb,36,2
3,app/services/post_status_service.rb,162,5
4,app/services/vote_service.rb,62,3
...,...,...,...
810,app/policies/account_warning_policy.rb,13,2
811,app/policies/ip_block_policy.rb,12,0
812,app/policies/announcement_policy.rb,15,0
813,app/policies/application_policy.rb,16,0


In [59]:
import plotly.express as px

fig = px.scatter(df_complexity, x="loc", y="changes",
                hover_data=['fullpath'])
fig.show()

> Analyze six candidate hotspots (not necessarily the top ones) through:  
>   (a) complexity trends analysis


I consider the following six candidates

In [60]:
candidates = ['app/models/status.rb','app/models/account.rb','app/lib/feed_manager.rb',
              'app/models/user.rb','app/helpers/application_helper.rb','app/helpers/languages_helper.rb']

rows = []

for commit in repo.traverse_commits():
    for m_file in commit.modified_files:
        if m_file.new_path in candidates:
            d = {}
            d['fullpath'] = m_file.new_path
            d['date'] = commit.committer_date
            d[loc] = m_file.nloc
            rows.append(d)
df = pd.DataFrame(rows)


In [61]:
fig = px.line(df, x='date', y='loc', color='fullpath')
fig.show()

we don't see much change here, probably due to short timeframe. I try a longer one.

In [63]:
older_tag = 'v3.2.0'

repo = pydriller.Repository(path_to_repo, from_tag = older_tag, to_tag = current_tag)

rows = []

for commit in repo.traverse_commits():
    for m_file in commit.modified_files:
        if m_file.new_path in candidates:
            d = {}
            d['fullpath'] = m_file.new_path
            d['date'] = commit.committer_date
            d[loc] = m_file.nloc
            rows.append(d)
df = pd.DataFrame(rows)

fig = px.line(df, x='date', y='loc', color='fullpath')
fig.show()

> Analyze six candidate hotspots (not necessarily the top ones) through:  
>   (b) manual analysis of entities names and content

In [64]:
from pygments import highlight
from pygments.lexers import RubyLexer
from pygments.formatters import HtmlFormatter
import IPython

codes = []


for candidate in candidates:
    print(path_to_repo + candidate)
    with open(path_to_repo + '/' + candidate) as f:
        codes.append(f.read())



formatter = HtmlFormatter()


IPython.display.HTML('<style type="text/css">{}</style>{}'.format(
    formatter.get_style_defs('.highlight'),
    highlight(codes[0], RubyLexer(), formatter)))

# repeat for all entities, or use another tool to open them

./mastodonapp/models/status.rb
./mastodonapp/models/account.rb
./mastodonapp/lib/feed_manager.rb
./mastodonapp/models/user.rb
./mastodonapp/helpers/application_helper.rb
./mastodonapp/helpers/languages_helper.rb


## 2. Temporal/Logical Coupling

> Determine what could be cases of temporal/logical coupling and generate a list of candidates with a set of coupled entities.

I consider only the entities within the version of interest (tag 3.5.3). I also focus only on pairs.

In [65]:
from collections import Counter
import itertools

co_changes = []

repo = pydriller.Repository(path_to_repo, from_tag = start_tag, to_tag = current_tag)

for commit in repo.traverse_commits():
    changed_paths = []
    
    for m_file in commit.modified_files:
        if m_file.new_path in entities_paths:
            co_changes.append(m_file.new_path)
            changed_paths.append(m_file.new_path)
        
    changed_paths.sort()
    for cp in itertools.combinations(changed_paths,2):
        co_changes.append(cp[0] + '---' + cp[1])

c = Counter(co_changes)

c.most_common()

[('app/models/status.rb', 20),
 ('app/models/user.rb', 17),
 ('app/models/account.rb', 16),
 ('app/services/activitypub/process_status_update_service.rb', 15),
 ('app/lib/feed_manager.rb', 11),
 ('app/helpers/application_helper.rb', 10),
 ('app/services/remove_status_service.rb', 9),
 ('app/lib/activitypub/activity/create.rb', 9),
 ('app/controllers/auth/sessions_controller.rb', 8),
 ('app/services/notify_service.rb', 8),
 ('app/chewy/statuses_index.rb', 8),
 ('app/helpers/languages_helper.rb', 8),
 ('app/services/update_status_service.rb', 8),
 ('app/services/fetch_link_card_service.rb', 7),
 ('app/models/media_attachment.rb', 7),
 ('app/lib/link_details_extractor.rb', 7),
 ('app/lib/activitypub/activity/announce.rb', 7),
 ('app/lib/activitypub/activity.rb', 6),
 ('app/services/resolve_account_service.rb', 6),
 ('app/services/activitypub/fetch_featured_collection_service.rb', 6),
 ('app/models/admin/status_batch_action.rb', 6),
 ('app/services/fan_out_on_write_service.rb', 6),
 ('app/

In [66]:
df = pd.DataFrame.from_dict(c, orient='index').reset_index()
df = df.rename(columns={'index':'fullpath', 0:'count'})
df

,fullpath,count
0,app/lib/activitypub/activity.rb,6
1,app/services/activitypub/process_account_servi...,4
2,app/services/fetch_link_card_service.rb,7
3,app/services/remove_status_service.rb,9
4,app/services/resolve_account_service.rb,6
...,...,...
5211,app/controllers/auth/confirmations_controller....,1
5212,app/models/admin/status_batch_action.rb---app/...,1
5213,app/services/unfollow_service.rb,1
5214,app/helpers/formatting_helper.rb---app/helpers...,1


>Visualize these candidate sets of couple entitiese with a visualization of your choice.

In [67]:
fig = px.bar(df.nlargest(30,'count'), x="count", y="fullpath", orientation='h')
fig.show()

Let's just try to focus on *pairs*

In [68]:
c_pairs = {k: v for k, v in c.items() if '---' in k}

df_pairs = pd.DataFrame.from_dict(c_pairs, orient='index').reset_index()
df_pairs = df_pairs.rename(columns={'index':'fullpath', 0:'count'})

In [69]:
fig = px.bar(df_pairs.nlargest(30,'count'), x="count", y="fullpath", orientation='h')
fig.show()

## 3. Defective Hotspots

> Decide on how you want to detect entities that had defects in the past (e.g., commit message analysis vs. issue tracking system analysis) and motivate your choice.

For simplicity, since the developers of go-ipfs do not seem to use just the bug number, I am going to use the commit message to determine fixes.

>Determine defective hotspots among the entities in the timeframe that you previously selected (i.e., consider only defects in the selected timeframe). What conclusions can you draw from this?

In [70]:
metrics = dict()
loc = 'loc'
changes = 'changes'
defects = 'defects'

bugs_keywords = ['fix','bug','error','mistake']

for entity_path in entities_paths:
    metrics[entity_path] = {loc : count_loc(entity_path), changes : 0, defects : 0}

repo = pydriller.Repository(path_to_repo, from_tag = start_tag, to_tag = current_tag)

for commit in repo.traverse_commits():
    for m_file in commit.modified_files:
        if m_file.new_path in entities_paths:
            metrics[m_file.new_path][changes] += 1
            if any(word in commit.msg for word in bugs_keywords):
                metrics[m_file.new_path][defects] += 1

csv_like = [('fullpath',loc,changes,defects)]

for fullpath,values in metrics.items():
    csv_like.append((fullpath,values[loc],values[changes],values[defects]))

In [71]:
df_defects = pd.DataFrame(csv_like[1:], columns=csv_like[0])
df_defects

,fullpath,loc,changes,defects
0,app/services/notify_service.rb,139,8,0
1,app/services/fan_out_on_write_service.rb,115,6,2
2,app/services/favourite_service.rb,36,2,1
3,app/services/post_status_service.rb,162,5,1
4,app/services/vote_service.rb,62,3,0
...,...,...,...,...
810,app/policies/account_warning_policy.rb,13,2,2
811,app/policies/ip_block_policy.rb,12,0,0
812,app/policies/announcement_policy.rb,15,0,0
813,app/policies/application_policy.rb,16,0,0


In [72]:
fig = px.scatter(df_defects, x="loc", y="changes", color="defects", size="defects",
                hover_data=['fullpath'])
fig.show()

>Determine complexity hotspots at the beginning of your timeframe, then correlate them with the defects they have presented throughout the entire timeframe. Is there a correlation? Why do you think this is the case?

In [73]:
metrics = dict()
past_loc = 'past_loc'
past_changes = 'past_changes'
defects = 'defects'


bugs_keywords = ['fix','bug','error','mistake']

# collect previous metrics
gr.checkout(older_tag)

for entity_path in entities_paths:
    metrics[entity_path] = {past_loc : count_loc(entity_path), past_changes : 0, defects : 0}


repo = pydriller.Repository(path_to_repo, from_tag = older_tag, to_tag = start_tag)

for commit in repo.traverse_commits():
    for m_file in commit.modified_files:
        if m_file.new_path in entities_paths:
            metrics[m_file.new_path][past_changes] += 1

# collect following metrics
gr.checkout(current_tag)

repo = pydriller.Repository(path_to_repo, from_tag = start_tag, to_tag = current_tag)

for commit in repo.traverse_commits():
    for m_file in commit.modified_files:
        if m_file.new_path in entities_paths and any(word in commit.msg for word in bugs_keywords):
            metrics[m_file.new_path][defects] += 1


In [74]:
csv_like = [('fullpath',past_loc,past_changes,defects)]

for fullpath,values in metrics.items():
    csv_like.append((fullpath,values[past_loc],values[past_changes],values[defects]))

df_futuredefects = pd.DataFrame(csv_like[1:], columns=csv_like[0])

fig = px.scatter(df_futuredefects, x=past_loc, y=past_changes, color="defects", size="defects",
                hover_data=['fullpath'])
fig.show()